In [14]:
import cv2
import numpy as np

In [15]:
def resizeImage(img, percent=100):
    scale_percent = percent # percent of original size
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    # resize image
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
    return resized

In [16]:
# Load the shape template or reference image
template = cv2.imread('./quochuy2_shape.jpg')
template = resizeImage(template, 30)
template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

cv2.imshow('Template', template)
cv2.waitKey()
cv2.destroyWindow('Template')

In [42]:
template_edges = cv2.Canny(template_gray, 50, 120)
cv2.imshow('template_edges', template_edges)
cv2.waitKey(0)
cv2.destroyWindow('template_edges')

In [18]:
# Load the target image with the shapes we're trying to match
target = cv2.imread('data/1.jpg')
target = resizeImage(target, 30)
target_gray = cv2.cvtColor(target, cv2.COLOR_BGR2GRAY)

cv2.imshow('Target', target_gray)
cv2.waitKey()
cv2.destroyWindow('Target')

In [49]:
target_edges = cv2.Canny(target_gray, 50, 150)
cv2.imshow('target_edges', target_edges)
cv2.waitKey(0)
cv2.destroyWindow('target_edges')

In [19]:
# Threshold image first before using cv2.findContours
ret, thresh1 = cv2.threshold(template_gray, 127, 255, 0)
# Find contours in template
# contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
contours, hierarchy = cv2.findContours(template_edges, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
print("Length contours: ", len(contours))


Length contours:  31


There are two ways to get largest contour

In [29]:
# Way 1
# We need to sort the contours by area so that we can remove the largest
# contour which is the image outline
sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)
# We extract the second largest contour which will be our template contour
template_contour = sorted_contours[0]

In [23]:
# Way 2
# put to array then get max
areas = []
for contour in contours:
  ar = cv2.contourArea(contour)
  areas.append(ar)

max_area = max(areas)
max_area_index = areas.index(max_area) #index of the list element with largest area

cnt = contours[max_area_index] #largest area contour

In [13]:
# template = cv2.imread('./quochuy2_shape.jpg')
# cv2.drawContours(template, cnt, 0, (0, 255, 0), 3, max_level=0)

cv2.drawContours(template_gray, [cnt], 0, (0, 40, 120), 3, maxLevel = 0)

cv2.imshow('Template_Contours', template_gray)
cv2.waitKey(0)
cv2.destroyWindow('Template_Contours')

In [50]:
import sys

# Find contour of target image
ret, thresh2 = cv2.threshold(target_gray, 127, 255, 0)

# Extract contours from second target image
# contours, hierarchy = cv2.findContours(thresh2, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
contours, hierarchy = cv2.findContours(target_edges, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

min_match = sys.maxsize
closest_contour = []
for c in contours:
    # Iterate through each contour in the target image and
    # use cv2.matchShapes to compare contour shapes
    match = cv2.matchShapes(template_contour, c, 3, 0.0)
#     print(match)
    if match < min_match:
        min_match = match
        closest_contour = c
    # If the match value is less than 0.15 we
#     if match < 0.15:
#         closest_contour = c
#     else:
#         closest_contour = []
        
print(min_match)
cv2.drawContours(target, [closest_contour], -1, (0, 255, 0), 3)
cv2.imshow('Target_contour', target)
cv2.waitKey()
cv2.destroyWindow('Target_contour')

0.8027191089317323


In [ ]:
# Threshold both images first before using cv2.findContours
ret, thresh1 = cv2.threshold(template, 127, 255, 0)
ret, thresh2 = cv2.threshold(target_gray, 127, 255, 0)

# Find contours in template
contours, hierarchy = cv2.findContours(thresh1, cv2.RETR_CCOMP,
                                       cv2.CHAIN_APPROX_SIMPLE)

# We need to sort the contours by area so that we can remove the largest
# contour which is the image outline
sorted_contours = sorted(contours, key=cv2.contourArea, reverse=True)

# We extract the second largest contour which will be our template contour
template_contour = contours[0]

# Extract contours from second target image
contours, hierarchy = cv2.findContours(thresh2, cv2.RETR_CCOMP,
                                       cv2.CHAIN_APPROX_SIMPLE)

for c in contours:
    # Iterate through each contour in the target image and
    # use cv2.matchShapes to compare contour shapes
    match = cv2.matchShapes(template_contour, c, 3, 0.0)
    #     print(match)
    # If the match value is less than 0.15 we
    if match < 0.15:
        closest_contour = c
    else:
        closest_contour = []

cv2.drawContours(target, [closest_contour], -1, (0, 255, 0), 3)
cv2.imshow('Output', target)
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
cv2.destroyAllWindows()

In [55]:

method = cv2.TM_SQDIFF_NORMED

# Read the images from the file
pic = cv2.imread('quoc_huy.jpg',0)
# small_image = template_gray
small_image = resizeImage(pic, 30)
# large_image = cv2.imread('data/1.jpg')
large_image = target_gray

result = cv2.matchTemplate(small_image, large_image, method)
# print(result)
# We want the minimum squared difference
min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
print(cv2.minMaxLoc(result))
# Draw the rectangle:
# Extract the coordinates of our best match
MPx,MPy = min_loc

# Step 2: Get the size of the template. This is the same size as the match.
trows,tcols = small_image.shape[:2]

# Step 3: Draw the rectangle on large_image
cv2.rectangle(large_image, (MPx,MPy),(MPx+tcols,MPy+trows),(0,0,255),2)

# Display the original image with the rectangle around the match.
cv2.imshow('output',large_image)

# The image is only displayed if we call this
cv2.waitKey(0)
cv2.destroyWindow('output')

(0.2910112738609314, 0.9531067609786987, (135, 273), (88, 218))
